# Cleaning and Structuring the Data

## **Introduction**
Your manager is impressed with your progress but points out that the data is messy. Before we can analyze it effectively, we need to **clean and structure the data** properly.

Your task is to:
- Handle missing values
- Remove duplicate or inconsistent data
- Standardize the data format

Let's get started!

---

## **Task 1: Identify Issues in the Data**
Your manager provides you with an example dataset where some records are incomplete or incorrect. Here’s an example:

```json
{
    "users": [
        {"id": 1, "name": "Amit", "friends": [2, 3], "liked_pages": [101]},
        {"id": 2, "name": "Priya", "friends": [1, 4], "liked_pages": [102]},
        {"id": 3, "name": "", "friends": [1], "liked_pages": [101, 103]},
        {"id": 4, "name": "Sara", "friends": [2, 2], "liked_pages": [104]},
        {"id": 5, "name": "Amit", "friends": [], "liked_pages": []}
    ],
    "pages": [
        {"id": 101, "name": "Python Developers"},
        {"id": 102, "name": "Data Science Enthusiasts"},
        {"id": 103, "name": "AI & ML Community"},
        {"id": 104, "name": "Web Dev Hub"},
        {"id": 104, "name": "Web Development"}
    ]
}
```

**Problems:**
1. User **ID 3** has an empty name.
2. User **ID 4** has a duplicate friend entry.
3. User **ID 5** has no connections or liked pages (inactive user).
4. The **pages list** contains duplicate page IDs.

---

## **Task 2: Clean the Data**
We will:
1. Remove users with missing names.
2. Remove duplicate friend entries.
3. Remove inactive users (users with no friends and no liked pages).
4. Deduplicate pages based on IDs.

### **Code Implementation**

```python
import json

def clean_data(data):
    # Remove users with missing names
    data["users"] = [user for user in data["users"] if user["name"].strip()]
    
    # Remove duplicate friends
    for user in data["users"]:
        user["friends"] = list(set(user["friends"]))
    
    # Remove inactive users
    data["users"] = [user for user in data["users"] if user["friends"] or user["liked_pages"]]
    
    # Remove duplicate pages
    unique_pages = {}
    for page in data["pages"]:
        unique_pages[page["id"]] = page
    data["pages"] = list(unique_pages.values())
    
    return data

# Load, clean, and display the cleaned data
data = json.load(open("codebook_data.json"))
data = clean_data(data)
json.dump(data, open("cleaned_codebook_data.json", "w"), indent=4)
print("Data cleaned successfully!")
```

---

## **Expected Output:**
The cleaned dataset will:
- Remove users with missing names

- Ensure friend lists contain unique entries

- Remove inactive users

- Deduplicate pages


---

## **Next Steps**
Your manager is happy with the cleaned data and says: **"Great! Now that our data is structured, let's start analyzing it. First, let's build a 'People You May Know' feature!"**

In [7]:
import json
def clean_data(data):
        # Remove users with missing names
    data['users'] = [user for user in data['users'] if user['name'].strip() != ""]
    
    # Remove inactive users
    data['users'] = [user for user in data['users'] if user['friends'] != [] or user['liked_pages'] != []]

       # Remove duplicate friends
    for user in data['users']:
        user['friends']=list(set(user['friends']))

       # Remove duplicate pages
    uniquepages = {}
    for page in data['pages']:
        uniquepages[page['id']] = page
    data['pages'] = list(uniquepages.values())
        
    return data
    
# Load, clean, and display the cleaned data
data = json.load(open("data_2.json","r"))
data = clean_data(data)
json.dump(data,open("cleaned_data2.json","w"),indent = 4)
print("data has been cleaned sucessfully")

data has been cleaned sucessfully


# using pandas

In [3]:
import pandas as pd

import json

with open('data_2.json','r') as f:
    data = json.load(f)

In [4]:
data

{'users': [{'id': 1, 'name': 'Amit', 'friends': [2, 3], 'liked_pages': [101]},
  {'id': 2, 'name': 'Priya', 'friends': [1, 4], 'liked_pages': [102]},
  {'id': 3, 'name': '', 'friends': [1], 'liked_pages': [101, 103]},
  {'id': 4, 'name': 'Sara', 'friends': [2, 2], 'liked_pages': [104]},
  {'id': 5, 'name': 'Amit', 'friends': [], 'liked_pages': []}],
 'pages': [{'id': 101, 'name': 'Python Developers'},
  {'id': 102, 'name': 'Data Science Enthusiasts'},
  {'id': 103, 'name': 'AI & ML Community'},
  {'id': 104, 'name': 'Web Dev Hub'},
  {'id': 104, 'name': 'Web Development'}]}

In [5]:
df1 = pd.json_normalize(data['users'])

In [6]:
df1

,id,name,friends,liked_pages
0,1,Amit,"[2, 3]",[101]
1,2,Priya,"[1, 4]",[102]
2,3,,[1],"[101, 103]"
3,4,Sara,"[2, 2]",[104]
4,5,Amit,[],[]


In [7]:
df2 = pd.json_normalize(data['pages'])

In [8]:
df2

,id,name
0,101,Python Developers
1,102,Data Science Enthusiasts
2,103,AI & ML Community
3,104,Web Dev Hub
4,104,Web Development


In [11]:
df3 = df1[df1['name']!= ""].copy()

In [27]:
df3 = df3[df3.apply(lambda x:x['friends'] != [] and x['liked_pages'] != [],axis = 1)]

In [28]:
df3

,id,name,friends,liked_pages
0,1,Amit,"[2, 3]",[101]
1,2,Priya,"[1, 4]",[102]
3,4,Sara,"[2, 2]",[104]


In [29]:
df2

,id,name
0,101,Python Developers
1,102,Data Science Enthusiasts
2,103,AI & ML Community
3,104,Web Dev Hub
4,104,Web Development


In [35]:
df4 = df2.drop_duplicates(subset = ['id']).copy()

In [36]:
df4

,id,name
0,101,Python Developers
1,102,Data Science Enthusiasts
2,103,AI & ML Community
3,104,Web Dev Hub


In [37]:
with pd.ExcelWriter("report.xlsx") as writer:
    df3.to_excel(writer, sheet_name="users", index=False)
    df4.to_excel(writer, sheet_name="pages", index=False)